In [ ]:
import json
import pandas
import pyarrow
from sqlalchemy import create_engine

# Steps to install
# 1. pip install google-cloud-bigquery-storage pyarrow matplotlib sqlalchemy-bigquery
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [ ]:
COMPANY_IDENTIFER = 'HPCC'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

company_total_sales_transactions_count_query = f"""
    select
        count(*) as total_count
    from
        companies
        inner join metrc_sales_receipts on companies.id = metrc_sales_receipts.company_id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = '{COMPANY_IDENTIFER}'
"""

company_count_incoming_transfer_packages_query = f"""
    select
        date_trunc(metrc_transfers.created_date, month) as date_type,
        metrc_transfer_packages.product_category_name,
        count(metrc_transfer_packages.id) as count
    from
        company_deliveries
        inner join companies on company_deliveries.company_id = companies.id
        inner join metrc_transfers on company_deliveries.transfer_row_id = metrc_transfers.id
        inner join metrc_deliveries on metrc_transfers.id = metrc_deliveries.transfer_row_id
        inner join metrc_transfer_packages on metrc_deliveries.id = metrc_transfer_packages.delivery_row_id
    where
        True
        and companies.identifier = '{COMPANY_IDENTIFER}'
        and metrc_transfers.created_date >= '{SALES_TRANSACTIONS_START_DATE}'
        and (
            company_deliveries.delivery_type = 'INCOMING_FROM_VENDOR' or
            company_deliveries.delivery_type = 'INCOMING_UNKNOWN'
        )
    group by
        1,
        2
    order by
        1 asc,
        2 asc
"""

company_total_gmv_from_receipts_query = f"""
    select
        date_trunc(metrc_sales_receipts.sales_datetime, month) as date_type,
        sum(metrc_sales_receipts.total_price) as total_gmv
    from
        companies
        inner join metrc_sales_receipts on companies.id = metrc_sales_receipts.company_id
    where
        True
        and companies.identifier = '{COMPANY_IDENTIFER}'
        and metrc_sales_receipts.sales_datetime >= '{SALES_TRANSACTIONS_START_DATE}'
    group by
        1
    order by
        1 asc
"""

company_total_gmv_from_transactions_query = f"""
    select
        date_trunc(metrc_sales_receipts.sales_datetime, month) as date_type,
        metrc_sales_transactions.product_category_name,
        sum(metrc_sales_transactions.total_price) as total_gmv
    from
        companies
        inner join metrc_sales_receipts on companies.id = metrc_sales_receipts.company_id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = '{COMPANY_IDENTIFER}'
        and metrc_sales_receipts.sales_datetime >= '{SALES_TRANSACTIONS_START_DATE}'
    group by
        1,
        2
    order by
        1 asc,
        2 asc
"""

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
# test = pandas.read_sql_query(incoming_transfer_packages_query, engine)
# company_total_sales_transactions_count_dataframe = pandas.read_sql_query(company_total_sales_transactions_count_query, engine)
# company_count_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_count_incoming_transfer_packages_query, engine)
company_total_gmv_from_receipts_dataframe = pandas.read_sql_query(company_total_gmv_from_receipts_query, engine)
company_total_gmv_from_transactions_dataframe = pandas.read_sql_query(company_total_gmv_query, engine)

## Count sales transactions all time

In [ ]:
company_total_sales_transactions_count_dataframe

## Count incoming transfer packages by date

In [ ]:
company_count_incoming_transfer_packages_dataframe

In [ ]:
company_count_incoming_transfer_packages_dataframe.groupby(['date_type'])['count'].sum()

In [ ]:
company_count_incoming_transfer_packages_dataframe.groupby(['date_type', 'product_category_name'])['count'].sum().unstack().plot.bar(figsize=(24, 8), stacked=True)

## Total GMV from sales receipts by date

In [ ]:
company_total_gmv_from_receipts_dataframe

In [ ]:
company_total_gmv_from_receipts_dataframe['total_gmv'].plot.bar(figsize=(24, 8), stacked=True)

In [ ]:
company_total_gmv_from_transactions_dataframe

In [ ]:
company_total_gmv_from_transactions_dataframe.groupby(['date_type'])['total_gmv'].sum()

In [ ]:
company_total_gmv_from_transactions_dataframe.groupby(['date_type', 'product_category_name'])['total_gmv'].sum().unstack().plot.bar(figsize=(24, 8), stacked=True)